An incomplete imitiation of what PHP's composer does, and what https://php-download.com/ probably does

The packagist API is documented at https://packagist.org/apidoc#get-package-data

In [1]:
import requests
import json

# before getting to the function, try an example 

In [10]:
r = requests.get("https://repo.packagist.org/p/mediawiki/semantic-media-wiki.json")
data = json.loads(r.text)

print("where to get the source code:")
data['packages']['mediawiki/semantic-media-wiki']['3.2.2']['source']

where to get the source code:


{'type': 'git',
 'url': 'https://github.com/SemanticMediaWiki/SemanticMediaWiki.git',
 'reference': 'b75e296a640f9cf6d28526225bf87e63c2a951f0'}

In [11]:
print("dependencies:")
data['packages']['mediawiki/semantic-media-wiki']['3.2.2']['require']

dependencies:


{'php': '>=7.1',
 'ext-mbstring': '*',
 'composer/installers': '1.*,>=1.0.1',
 'psr/log': '~1.0',
 'mediawiki/parser-hooks': '~1.4',
 'param-processor/param-processor': '~1.2',
 'serialization/serialization': '~3.2|~4.0',
 'onoi/message-reporter': '~1.0',
 'onoi/cache': '~1.2',
 'onoi/event-dispatcher': '~1.0',
 'onoi/blob-store': '~1.2',
 'onoi/http-request': '~1.1',
 'onoi/callback-container': '~2.0',
 'onoi/tesa': '~0.1',
 'onoi/shared-resources': '~0.3',
 'symfony/css-selector': '^3.3',
 'elasticsearch/elasticsearch': '^5.3|^6.0',
 'seld/jsonlint': '^1.7',
 'justinrainbow/json-schema': '~5.2'}

for each of those packages, repeat the request

# now as recursive function

The code below would need to be modified to deal with the variations in the list of versions per package

In [2]:
def get_version_string(version: str) -> str:
    """
    This function cleans up strings of the form
    
    >=7.1
    *
    1.*,>=1.0.1
    ~1.0
    ~3.2|~4.0
    ^5.3|^6.0
    """
    if "," in version:
        ver = version.split(",")[-1]
    elif "|" in version:
        ver = version.split("|")[-1]
    else:
        ver = version
    print(ver)    
    ver = ver.replace("~","").replace("^","").replace(">=","")
    print(ver)
    return ver

In [3]:
def get_source_and_dependencies(list_of_src: list, vendor_package: str, version: str):
    """
    recursive function to resolved PHP package dependencies
    
    """
    if vendor_package=="php":
        return
    print("for "+vendor_package+"; version "+version)
    r = requests.get("https://repo.packagist.org/p/"+vendor_package+".json")
    if (r.status_code==404):
        print("URL not found; going to next\n")
        return
    data = json.loads(r.text)
    print("\n  versions:")
    print(data['packages'][vendor_package].keys())
    if version in list(data['packages'][vendor_package].keys()):
        print("\n  source:")
        print(data['packages'][vendor_package][version]['source'])
        list_of_src.append(data['packages'][vendor_package][version]['source'])
    
        print("\n  dependencies:")
        for dependency, ver in data['packages'][vendor_package][version]['require'].items():
            print(dependency," ",ver)
            get_source_and_dependencies(list_of_src, dependency, get_version_string(ver))
    else:
        print("appropriate version "+version+" not found in list\n")

In [4]:
list_of_src = []
get_source_and_dependencies(list_of_src, 'mediawiki/semantic-media-wiki','3.2.2')

for mediawiki/semantic-media-wiki; version 3.2.2

  versions:
dict_keys(['1.9', '1.9-RC1', '1.9.0.1', '1.9.0.2', '1.9.1', '1.9.1.1', '1.9.2', '1.9beta1', '2.0', '2.0-RC1', '2.0-RC2', '2.0-RC3', '2.0.x-dev', '2.1.0', '2.1.0-RC1', '2.1.1', '2.1.2', '2.1.3', '2.1.x-dev', '2.2.0', '2.2.0-RC1', '2.2.1', '2.2.2', '2.2.3', '2.2.x-dev', '2.3.0', '2.3.0-RC1', '2.3.1', '2.3.x-dev', '2.4.0', '2.4.0-RC1', '2.4.1', '2.4.2', '2.4.3', '2.4.4', '2.4.5', '2.4.6', '2.4.x-dev', '2.5.0', '2.5.0-rc.1', '2.5.1', '2.5.2', '2.5.3', '2.5.4', '2.5.5', '2.5.6', '2.5.7', '2.5.8', '2.5.x-dev', '3.0.0', '3.0.0-rc.1', '3.0.0-rc.2', '3.0.1', '3.0.2', '3.0.x-dev', '3.1.0', '3.1.0-rc.1', '3.1.1', '3.1.2', '3.1.3', '3.1.4', '3.1.5', '3.1.6', '3.1.x-dev', '3.2.0', '3.2.0-rc.1', '3.2.0-rc.2', '3.2.1', '3.2.2', '3.2.x-dev', 'dev-135x-testing', 'dev-320rc1', 'dev-ConnRef', 'dev-ContentParser', 'dev-call-func', 'dev-check-prep', 'dev-compat', 'dev-composer-load', 'dev-coverage', 'dev-encode', 'dev-faceted-search', 'dev-issue

In [7]:
list_of_src

[{'type': 'git',
  'url': 'https://github.com/SemanticMediaWiki/SemanticMediaWiki.git',
  'reference': 'b75e296a640f9cf6d28526225bf87e63c2a951f0'},
 {'type': 'git',
  'url': 'https://github.com/JeroenDeDauw/ParserHooks.git',
  'reference': 'a088e150ab8876c1b4b204f3b7930231d1c0113a'},
 {'type': 'git',
  'url': 'https://github.com/JeroenDeDauw/Validator.git',
  'reference': '57c6f20f8c0100865e4318b9e5277dd2c4331a0a'},
 {'type': 'git',
  'url': 'https://github.com/JeroenDeDauw/ParamProcessor.git',
  'reference': '14f88e2a9b4e82fe48510c3fb156a01b0aea3266'},
 {'type': 'git',
  'url': 'https://github.com/DataValues/DataValues.git',
  'reference': '603a19faeac34f5db23b786c6d9c20a151118c2a'},
 {'type': 'git',
  'url': 'https://github.com/DataValues/Interfaces.git',
  'reference': '38a04aede894929d822001e5e8851f4a08454a89'},
 {'type': 'git',
  'url': 'https://github.com/DataValues/Common.git',
  'reference': '406663b2a68bcf88917d685ab53eb383ff60a56c'},
 {'type': 'git',
  'url': 'https://github.